# Classical Optimization Methods for Quantum Problems

## Learning Objectives
- Implement enhanced simulated annealing with adaptive cooling
- Learn problem-specific local search heuristics
- Compare classical methods with quantum approaches
- Understand when to use classical vs quantum methods

## Why Classical Methods Matter

Classical optimization methods remain crucial because:
1. **Benchmarking**: Compare quantum algorithm performance
2. **Hybrid approaches**: Classical preprocessing/postprocessing
3. **Current practicality**: Often outperform NISQ devices
4. **Problem insights**: Understanding optimization landscapes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Callable, Optional
import time
from dataclasses import dataclass
from abc import ABC, abstractmethod
import random
from collections import defaultdict

# Set random seeds
np.random.seed(42)
random.seed(42)

print("Classical Optimization Tutorial Environment Ready!")

## 1. Enhanced Simulated Annealing

Simulated annealing mimics the physical process of cooling metals. Key components:

1. **Temperature schedule**: Controls exploration vs exploitation
2. **Acceptance probability**: $P = e^{-\Delta E / T}$ for worse solutions
3. **Neighborhood structure**: How to generate new solutions
4. **Adaptive mechanisms**: Adjust parameters during search

In [ ]:
@dataclass
class OptimizationResult:
    """Store optimization results and statistics"""
    best_solution: np.ndarray
    best_objective: float
    iterations: int
    convergence_history: List[float]
    temperature_history: List[float]
    acceptance_rate: float
    runtime_seconds: float

class CoolingSchedule(ABC):
    """Abstract base class for cooling schedules"""
    
    @abstractmethod
    def get_temperature(self, iteration: int, initial_temp: float) -> float:
        pass

class ExponentialCooling(CoolingSchedule):
    """Exponential cooling: T(t) = T0 * alpha^t"""
    
    def __init__(self, alpha: float = 0.95):
        self.alpha = alpha
    
    def get_temperature(self, iteration: int, initial_temp: float) -> float:
        return initial_temp * (self.alpha ** iteration)

class LinearCooling(CoolingSchedule):
    """Linear cooling: T(t) = T0 * (1 - t/max_iter)"""
    
    def __init__(self, max_iterations: int):
        self.max_iterations = max_iterations
    
    def get_temperature(self, iteration: int, initial_temp: float) -> float:
        return initial_temp * max(0, 1 - iteration / self.max_iterations)

class AdaptiveCooling(CoolingSchedule):
    """Adaptive cooling based on acceptance rate"""
    
    def __init__(self, target_acceptance: float = 0.4, 
                 adaptation_rate: float = 0.1):
        self.target_acceptance = target_acceptance
        self.adaptation_rate = adaptation_rate
        self.current_temp_factor = 1.0
    
    def get_temperature(self, iteration: int, initial_temp: float) -> float:
        return initial_temp * self.current_temp_factor
    
    def update_temperature_factor(self, acceptance_rate: float):
        """Update temperature based on recent acceptance rate"""
        if acceptance_rate > self.target_acceptance:
            # Too many acceptances, cool down faster
            self.current_temp_factor *= (1 - self.adaptation_rate)
        else:
            # Too few acceptances, slow down cooling
            self.current_temp_factor *= (1 + self.adaptation_rate * 0.5)

print("Cooling schedule classes defined!")

In [ ]:
class EnhancedSimulatedAnnealing:
    """Enhanced simulated annealing with adaptive features"""
    
    def __init__(self, 
                 objective_function: Callable[[np.ndarray], float],
                 neighborhood_function: Callable[[np.ndarray], np.ndarray],
                 cooling_schedule: CoolingSchedule,
                 initial_temperature: float = 100.0,
                 max_iterations: int = 10000,
                 min_temperature: float = 0.01,
                 adaptation_window: int = 100):
        
        self.objective_function = objective_function
        self.neighborhood_function = neighborhood_function
        self.cooling_schedule = cooling_schedule
        self.initial_temperature = initial_temperature
        self.max_iterations = max_iterations
        self.min_temperature = min_temperature
        self.adaptation_window = adaptation_window
    
    def optimize(self, initial_solution: np.ndarray) -> OptimizationResult:
        """Run simulated annealing optimization"""
        start_time = time.time()
        
        # Initialize
        current_solution = initial_solution.copy()
        current_objective = self.objective_function(current_solution)
        
        best_solution = current_solution.copy()
        best_objective = current_objective
        
        # Tracking
        convergence_history = [best_objective]
        temperature_history = [self.initial_temperature]
        recent_acceptances = []
        
        for iteration in range(self.max_iterations):
            # Get current temperature
            temperature = self.cooling_schedule.get_temperature(
                iteration, self.initial_temperature)
            
            if temperature < self.min_temperature:
                break
            
            # Generate neighbor
            neighbor = self.neighborhood_function(current_solution)
            neighbor_objective = self.objective_function(neighbor)
            
            # Accept or reject
            delta = neighbor_objective - current_objective
            
            if delta < 0:  # Better solution
                accept = True
            else:  # Worse solution, accept with probability
                accept_prob = np.exp(-delta / temperature)
                accept = np.random.random() < accept_prob
            
            recent_acceptances.append(accept)
            if len(recent_acceptances) > self.adaptation_window:
                recent_acceptances.pop(0)
            
            if accept:
                current_solution = neighbor
                current_objective = neighbor_objective
                
                # Track best solution
                if current_objective < best_objective:
                    best_solution = current_solution.copy()
                    best_objective = current_objective
            
            # Adaptive cooling update
            if (isinstance(self.cooling_schedule, AdaptiveCooling) and 
                len(recent_acceptances) >= self.adaptation_window):
                acceptance_rate = np.mean(recent_acceptances)
                self.cooling_schedule.update_temperature_factor(acceptance_rate)
            
            # Record progress
            convergence_history.append(best_objective)
            temperature_history.append(temperature)
        
        runtime = time.time() - start_time
        final_acceptance_rate = np.mean(recent_acceptances) if recent_acceptances else 0
        
        return OptimizationResult(
            best_solution=best_solution,
            best_objective=best_objective,
            iterations=iteration + 1,
            convergence_history=convergence_history,
            temperature_history=temperature_history,
            acceptance_rate=final_acceptance_rate,
            runtime_seconds=runtime
        )

print("Enhanced Simulated Annealing class ready!")

## 2. Problem-Specific Neighborhoods

The neighborhood function determines how we explore the solution space. Different problems need different neighborhood structures.

In [ ]:
class NeighborhoodFunctions:
    """Collection of neighborhood functions for different problem types"""
    
    @staticmethod
    def single_bit_flip(solution: np.ndarray) -> np.ndarray:
        """Flip a single random bit"""
        neighbor = solution.copy()
        idx = np.random.randint(len(solution))
        neighbor[idx] = 1 - neighbor[idx]  # Flip bit
        return neighbor
    
    @staticmethod
    def k_bit_flip(k: int = 2):
        """Flip k random bits"""
        def neighborhood_func(solution: np.ndarray) -> np.ndarray:
            neighbor = solution.copy()
            indices = np.random.choice(len(solution), size=k, replace=False)
            for idx in indices:
                neighbor[idx] = 1 - neighbor[idx]
            return neighbor
        return neighborhood_func
    
    @staticmethod
    def swap_bits(solution: np.ndarray) -> np.ndarray:
        """Swap values of two random positions"""
        neighbor = solution.copy()
        idx1, idx2 = np.random.choice(len(solution), size=2, replace=False)
        neighbor[idx1], neighbor[idx2] = neighbor[idx2], neighbor[idx1]
        return neighbor
    
    @staticmethod
    def guided_flip(objective_function: Callable[[np.ndarray], float], 
                   bias_strength: float = 0.7):
        """Bias bit flips toward improving moves"""
        def neighborhood_func(solution: np.ndarray) -> np.ndarray:
            if np.random.random() < bias_strength:
                # Try to make improving move
                current_obj = objective_function(solution)
                best_neighbor = None
                best_obj = current_obj
                
                # Try flipping each bit and pick best improvement
                for i in range(min(len(solution), 10)):  # Limit search
                    idx = np.random.randint(len(solution))
                    neighbor = solution.copy()
                    neighbor[idx] = 1 - neighbor[idx]
                    obj = objective_function(neighbor)
                    
                    if obj < best_obj:
                        best_neighbor = neighbor
                        best_obj = obj
                
                if best_neighbor is not None:
                    return best_neighbor
            
            # Fallback to random flip
            return NeighborhoodFunctions.single_bit_flip(solution)
        
        return neighborhood_func

print("Neighborhood functions defined!")

## 3. Practical Example: MaxCut Problem

Let's solve the MaxCut problem with different simulated annealing configurations and compare their performance.

In [ ]:
# MaxCut Problem Setup
def create_random_graph(n_nodes: int, edge_probability: float = 0.5, 
                       seed: int = 42) -> np.ndarray:
    """Create random weighted graph"""
    np.random.seed(seed)
    adj_matrix = np.zeros((n_nodes, n_nodes))
    
    for i in range(n_nodes):
        for j in range(i+1, n_nodes):
            if np.random.random() < edge_probability:
                weight = np.random.uniform(1, 10)
                adj_matrix[i, j] = weight
                adj_matrix[j, i] = weight
    
    return adj_matrix

def maxcut_objective(solution: np.ndarray, adj_matrix: np.ndarray) -> float:
    """MaxCut objective: minimize negative cut value"""
    cut_value = 0
    n = len(solution)
    
    for i in range(n):
        for j in range(i+1, n):
            if solution[i] != solution[j]:  # Different partitions
                cut_value += adj_matrix[i, j]
    
    return -cut_value  # Minimize negative (maximize cut)

def evaluate_cut_quality(solution: np.ndarray, adj_matrix: np.ndarray) -> dict:
    """Evaluate quality of a cut solution"""
    cut_value = -maxcut_objective(solution, adj_matrix)
    total_weight = np.sum(adj_matrix) / 2  # Each edge counted once
    cut_ratio = cut_value / total_weight if total_weight > 0 else 0
    
    partition_sizes = [np.sum(solution), len(solution) - np.sum(solution)]
    balance = min(partition_sizes) / max(partition_sizes) if max(partition_sizes) > 0 else 1
    
    return {
        'cut_value': cut_value,
        'cut_ratio': cut_ratio,
        'balance': balance,
        'partition_sizes': partition_sizes
    }

# Create test problem
n_nodes = 12
graph = create_random_graph(n_nodes, edge_probability=0.6)

print(f"Created random graph with {n_nodes} nodes")
print(f"Total edges: {np.sum(graph > 0) // 2}")
print(f"Total weight: {np.sum(graph) / 2:.2f}")

# Define objective function for this instance
def objective_func(x):
    return maxcut_objective(x, graph)

print("\nMaxCut problem setup complete!")